In [1]:
import sys
from pathlib import Path
import pandas as pd

# Add project root to Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

In [2]:
import src.load as load

sensor_locations_raw = load.load_sensor_locations()
sensor_counts_raw = load.load_sensor_counts()

In [3]:
import src.clean as clean


sensor_locations = clean.clean_sensor_locations(sensor_locations_raw)
sensor_counts = clean.clean_sensor_counts(sensor_counts_raw)
sensor_counts


,count_id,location_id,sensing_date,hourday,pedestriancount,sensor_name
0,1092020240921,109,2024-09-21,20,163,LatWill_T
1,1391220241109,139,2024-11-09,12,105,Signal1647_T
2,40720240923,40,2024-09-23,7,192,Spr201_T
3,211220241217,21,2024-12-17,12,872,Bourke155_T
4,871320250820,87,2025-08-20,13,306,Errol23_T
...,...,...,...,...,...,...
1449150,432020251217,43,2025-12-17,20,76,UM2_T
1449151,611320240511,61,2024-05-11,13,874,RMIT14_T
1449152,451620240208,45,2024-02-08,16,1185,Swa148_T
1449153,62020251102,6,2025-11-02,20,1494,FliS_T
